In [ ]:
from HeadscanData import * #Import all classes
import matplotlib.pyplot as plt
import proplot as pplt
import numpy as np
from tkinter.filedialog import askdirectory
import batchfile

import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

from HeadscanData import * #Import all classes
import proplot as pplt
import batchfile

In [ ]:
### file info ###
batchname = 'WaterfallExperiments'
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
calnames, scannames, datapath = eval('batchfile.' + batchname + '()') #batchfile.getscannames()
nsubjects = len(scannames)
datapath = os.path.expanduser(datapath)

numScansTot = 0
for scanName in scannames:
    numScansTot = numScansTot+len(scanName)

dataList = [{}] * numScansTot
subjectID = np.zeros(numScansTot)
ind = 0
### loop of subjects and scans
for ii in range(nsubjects):
    nscans = len(scannames[ii])
        #plt.close('all')
    for jj in range(nscans): 
        
        print('subject ' + str(ii+1) + ',  scan ' + str(jj+1))
    
        ### data and algo parameters ###
        calname = datapath + '/' + calnames[ii]
        scanname = datapath + '/' + scannames[ii][jj]
        
        #Testing new object oriented code
        scanParams  = HeadScanParams(scanname, calname)
        subjectScan = HeadScanData(scanParams)
        subjectScan.LoadScanDataAndPreprocess(cropTimeStart=1,cropTimeEnd=-1)
        subjectScan.RunWaveletFilteringOnSignal()
        if 1:#ii:
            subjectScan.GetPulseFeatures(lowNoise=True)
        else:
            subjectScan.GetPulseFeatures()
        dataList[ind] = subjectScan
        subjectID[ind] = ii+1
        ind = ind+1

In [ ]:
#print(dataList[11].scanData[0].contrast, np.min(dataList[11].scanData[0].contrast, axis=0))
array28Hz = ()
for i in range(11,len(dataList)):
    for scan in dataList[i].scanData:
        curArr = scan.contrast-np.percentile(scan.contrast,0.5,axis=0)
        curArr = curArr/np.percentile(curArr,99.5)
        curArr = np.expand_dims(curArr,axis=0)
        if not len(array28Hz):
            array28Hz = curArr[:,:,0::3]
        else:
            array28Hz = np.concatenate((array28Hz,curArr[:,:,0::3]))
array28Hz = np.clip(array28Hz,  0.0, 1.0)
print(array28Hz.shape, curArr.shape)

In [ ]:
#Data augmentation by

In [ ]:
def noise(array, noise_factor = 0.1):
    """
    Adds random noise to each image in the supplied array.
    """

    noisy_array = array + noise_factor * np.random.normal(
        loc=0.0, scale=1.0, size=array.shape
    )

    return np.clip(noisy_array, 0.0, 1.0)


def display(array1, array2, array3 = (), rowLabels = None ):
    """
    Displays ten random waveforms from each one of the supplied arrays.
    """

    n = 3

    indices = np.random.randint(len(array1), size=n)
    wave1 = array1[indices, :, :]
    wave2 = array2[indices, :, :]
    if len(array3):
        wave3 = array3[indices, :, :]
        fig, axs = pplt.subplots(ncols=3, nrows=3,refwidth=1.5)
    else:
        fig, axs = pplt.subplots(ncols=3, nrows=2,refwidth=1.5)
    axs.format(
        grid=False, xlabel='Samples', ylabel='Normalized Contrast',
        toplabels=('Scan 1', 'Scan 2', 'Scan 3'),
        leftlabels=rowLabels)

    for i, (wv1, wv2) in enumerate(zip(wave1, wave2)):
        axs[i].plot(wv1, labels=['Cam 1','Cam 4'])
        if not i:
            fig.legend()
        axs[i+n].plot(wv2, labels=['Cam 1','Cam 4'])
        if len(array3):
            axs[i+2*n].plot(np.squeeze(wave3[i,:,:]), labels=['Cam 4','Cam 4'])
    plt.show()

In [ ]:
# Since we only need images from the dataset to encode and decode, we
# won't use the labels.
train_data, test_data = train_test_split(array28Hz,test_size=0.1)

# Normalize and reshape the data
#train_data = preprocess(train_data)
#test_data = preprocess(test_data)

# Create a copy of the data with added noise
noisy_train_data = noise(train_data,noise_factor=0.01)
noisy_test_data = noise(test_data,noise_factor=0.01)

# Display the train data and a version of it with added noise
print(train_data.shape, test_data.shape)
print(noisy_train_data.shape, noisy_test_data.shape)
display(train_data, noisy_train_data, rowLabels=('Scan Data', 'Scan Data+Noise(Gaussian)'))

In [ ]:
input = layers.Input(shape=(200, 2, 1))

# Encoder
x = layers.Conv2D(32, (3, 1), activation="relu", padding="same")(input)
x = layers.Conv2D(32, (3, 1), activation="relu", padding="same", dilation_rate=2)(x)
x = layers.Conv2D(32, (3, 1), activation="relu", padding="same", dilation_rate=2)(x)
x = layers.Conv2D(32, (3, 1), activation="relu", padding="same", strides=(2,1))(x)
x = layers.Conv2D(32, (3, 1), activation="relu", padding="same")(x)
x = layers.Conv2D(32, (3, 1), activation="relu", padding="same", dilation_rate=2)(x)
x = layers.Conv2D(32, (3, 1), activation="relu", padding="same", dilation_rate=2)(x)
x = layers.Conv2D(32, (3, 1), activation="relu", padding="same", strides=(2,1))(x)
x = layers.Conv2D(32, (3, 1), activation="relu", padding="same")(x)
x = layers.Conv2D(32, (3, 1), activation="relu", padding="same", dilation_rate=2)(x)
x = layers.Conv2D(32, (3, 1), activation="relu", padding="same", dilation_rate=2)(x)
x = layers.Conv2D(32, (3, 1), activation="relu", padding="same", strides=(2,1))(x)
x = layers.Conv2D(32, (3, 1), activation="relu", padding="same", dilation_rate=2)(x)
x = layers.Conv2D(32, (3, 1), activation="relu", padding="same", dilation_rate=2)(x)
# Decoder
x = layers.Conv2DTranspose(32, (3, 1), activation="relu", padding="same", dilation_rate=2)(x)
x = layers.Conv2DTranspose(32, (3, 1), activation="relu", padding="same", dilation_rate=2)(x)
x = layers.Conv2DTranspose(32, (3, 1), strides=(2,1), activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(32, (3, 1), activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(32, (3, 1), activation="relu", padding="same", dilation_rate=2)(x)
x = layers.Conv2DTranspose(32, (3, 1), activation="relu", padding="same", dilation_rate=2)(x)
x = layers.Conv2DTranspose(32, (3, 1), strides=(2,1), activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(32, (3, 1), activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(32, (3, 1), activation="relu", padding="same", dilation_rate=2)(x)
x = layers.Conv2DTranspose(32, (3, 1), activation="relu", padding="same", dilation_rate=2)(x)
x = layers.Conv2DTranspose(32, (3, 1), strides=(2,1), activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(32, (3, 1), activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(32, (3, 1), activation="relu", padding="same", dilation_rate=2)(x)
x = layers.Conv2DTranspose(32, (3, 1), activation="relu", padding="same", dilation_rate=2)(x)
x = layers.Conv2D(1, (3, 1), activation="sigmoid", padding="same")(x)

# Autoencoder
autoencoder = Model(input, x)
autoencoder.compile(optimizer="adam", loss=tf.keras.losses.MeanSquaredError())#"binary_crossentropy")
autoencoder.summary()

In [ ]:
autoencoder.fit(
    x=train_data,
    y=train_data,
    epochs=100,
    batch_size=128,
    shuffle=True,
    validation_data=(test_data, test_data),
)

In [ ]:
predictions = autoencoder.predict(test_data)
print(predictions.shape)
display(test_data, np.squeeze(predictions), rowLabels=('Scan Data', 'Reconstructed Scan Data'))

In [ ]:
autoencoder.fit(
    x=noisy_train_data,
    y=train_data,
    epochs=100,
    batch_size=128,
    shuffle=True,
    validation_data=(noisy_test_data, test_data),
)

In [ ]:
predictions = autoencoder.predict(noisy_test_data)
display(noisy_test_data, np.squeeze(predictions), test_data, rowLabels=('Noisy', 'Reconstructed','Scan'))